* Time: Fall, 2019 
* Practice: STP 

Created to QuantEcon, Paralle Computing Repository [Practice 1](https://github.com/davidzarruk/Parallel_Computing/blob/master/R_main.R) and [Practice 2](https://github.com/davidzarruk/Parallel_Computing/blob/master/Rcpp_main.R)

### House Keeping

In [4]:
library("parallel")
args <- commandArgs(TRUE)

.libPaths( c( .libPaths(), "~/R/x86_64-pc-linux-gnu-library/3.4/") )

library("Rcpp")

setwd("/Users/apple/Desktop/bc_f19_econ")

Sys.setenv("PKG_CXXFLAGS"=" -fopenmp")

# Number of workers
#sourceCpp("Rcpp_main.cpp")

## Initialization 

In [8]:
# Number of workers
no_cores <- 5
#as.integer(args)
cl <- makeCluster(no_cores)

# Grid for x
nx            = 1500; 
xmin          = 0.1; 
xmax          = 4.0;

# Grid for e: parameters for Tauchen
ne            = 15; 
ssigma_eps    = 0.02058; 
llambda_eps   = 0.99; 
m             = 1.5; 

# Utility function
ssigma        = 2; 
bbeta         = 0.97;
T             = 10;

# Prices
r             = 0.07;
w             = 5;

# Initialize grids
xgrid = matrix(0, 1, nx)
egrid = matrix(0, 1, ne)
P     = matrix(0, ne, ne)
V     = array(0, dim=c(T, nx, ne))

## Grid Creation

In [9]:
# Grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for(i in 1:nx){
  xgrid[i] = xmin + it*xstep;
  it = it+1;
}

# Grid for productivity (e) with Tauchen (1986)
size = ne;
ssigma_y = sqrt((ssigma_eps^2) / (1 - (llambda_eps^2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for(i in 1:ne){
  egrid[i] = (-m*sqrt((ssigma_eps^2) / (1 - (llambda_eps^2))) + it*estep);
  it = it+1;
}

# Transition probability matrix (P) Tauchen (1986)
mm = egrid[2] - egrid[1];
for(j in 1:ne){
  for(k in 1:ne){
    if(k == 1){
      P[j, k] = pnorm((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
    } else if(k == ne){
      P[j, k] = 1 - pnorm((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
    } else{
      P[j, k] = pnorm((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - pnorm((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
    }
  }
}

# Exponential of the grid e
for(i in 1:ne){
  egrid[i] = exp(egrid[i]);
}

## Value Function I

In [ ]:
# Function that computes value function, given vector of state variables
value = function(ind){

  ix = as.integer(floor((ind-0.05)/ne))+1;
  ie = as.integer(floor((ind-0.05) %% ne)+1);
  
  VV = -10.0^3;
  for(ixp in 1:nx){
    
    expected = 0.0;
    if(age < T){
      for(iep in 1:ne){
        expected = expected + P[ie, iep]*V[age+1, ixp, iep];
      }
    }
    
    cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp];
    
    utility = (cons^(1-ssigma))/(1-ssigma) + bbeta*expected;
    
    if(cons <= 0){
      utility = -10.0^(5);
    }
    
    if(utility >= VV){
      VV = utility;
    }
  }
  
  return(VV);
}

## Value Function II

In [10]:
V = value(nx, xmin, xmax, 
      ne, ssigma_eps, llambda_eps, m, 
      ssigma, bbeta, T, r, w);


# I recover the Policy Functions
Value   = array(0,dim=c(T, nx, ne));

for (age in 1:T){
  for (ix in 1:nx){
    for(ie in 1:ne){
      Value[age, ix, ie]   = V[(age-1)*nx*ne + (ix-1)*ne + ie];
    }
  }
}

ERROR: Error in value(nx, xmin, xmax, ne, ssigma_eps, llambda_eps, m, ssigma, : could not find function "value"


## Life-cycle Computation 

In [11]:
print(" ")
print("Life cycle computation: ")
print(" ")

start = proc.time()[3];

for(age in T:1){

  clusterExport(cl, c("V", "age", "ne","nx", "r", "T", "P", "xgrid", "egrid", "ssigma", "bbeta", "w"), envir=environment()) 
  
  s = parLapply(cl, 1:(ne*nx), value)

  for(ind in 1:(nx*ne)){
    ix = as.integer(floor((ind-0.05)/ne))+1;
    ie = as.integer(floor((ind-0.05) %% ne)+1);
    
    V[age, ix, ie] = s[[ind]][1]
  }  
  
  finish = proc.time()[3] - start;
  print(paste0("Age: ", age, ". Time: ", round(finish, 3), " seconds."))
}

print(" ")
finish = proc.time()[3] - start;
print(paste("TOTAL ELAPSED TIME: ", finish, " seconds. "))

[1] " "
[1] "Life cycle computation: "
[1] " "


ERROR: Error in parLapply(cl, 1:(ne * nx), value): object 'value' not found


### Check

In [12]:
print(" ")
print(" - - - - - - - - - - - - - - - - - - - - - ")
print(" ")
print("The first entries of the value function: ")
print(" ")

for(i in 1:3){
  print(V[1, 1, i])
}

print(" ")

[1] " "
[1] " - - - - - - - - - - - - - - - - - - - - - "
[1] " "
[1] "The first entries of the value function: "
[1] " "
[1] 0
[1] 0
[1] 0
[1] " "


The end. 